In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import exp
#data = pd.read_csv("PerMinUnlabeledData.csv", delimiter = ',')
#data = data.drop(['Unnamed: 0','Min'],axis=1)
#wins = pd.read_csv("WinsForPerMinUnlabeledData.csv", delimiter = ',')
data = pd.read_csv("SHOT_DATA.csv", delimiter = ',')
FGM = data['FGM']
data = data.drop(['FGM','PTS','player_name'], axis = 1)

In [3]:
LAYER_1_SIZE = 10
LAYER_2_SIZE = 6
LAYER_3_SIZE = 4
def correct(x):
    for i in range(x.size):
        if x[i]<0:
            x[i]=0
    return x

def sigmoid(x):
    for i in range(x.size):
        x[i] = 1/(1+exp(-x[i]))
    return x

def sigmoidDerivative(x):
    inter = (1/(1+exp(-x)))*(1-(1/(1+exp(-x))))
    return inter

In [5]:
LAYER_1_WEIGHTS = (np.random.rand(LAYER_1_SIZE,x.size)*2)-1
LAYER_2_WEIGHTS = (np.random.rand(LAYER_2_SIZE,LAYER_1_SIZE)*2)-1
LAYER_3_WEIGHTS = (np.random.rand(LAYER_3_SIZE,LAYER_2_SIZE)*2)-1
OUTPUT_WEIGHTS = (np.random.rand(1,LAYER_3_SIZE)*2)-1

In [52]:
def predict(x):
    if(x[0].size!=1):
        print('input must be 1d array')
        return 0
    global LAYER_1
    global LAYER_2
    global LAYER_3
    global OUTPUT
    global LAYER_2_SIG
    global LAYER_3_SIG
    LAYER_1 = x*LAYER_1_WEIGHTS
    LAYER_1 = np.sum(LAYER_1,axis=1)
    LAYER_2 = LAYER_1*LAYER_2_WEIGHTS
    LAYER_2 = np.sum(LAYER_2,axis=1)
    LAYER_2_SIG = sigmoid(LAYER_2)
    LAYER_3 = LAYER_2_SIG*LAYER_3_WEIGHTS
    LAYER_3 = np.sum(LAYER_3,axis=1)
    LAYER_3_SIG = sigmoid(LAYER_3)
    OUTPUT = LAYER_3_SIG*OUTPUT_WEIGHTS
    OUTPUT = np.sum(OUTPUT,axis=1)
    return OUTPUT[0]

def error(input_array,expected_output):
    return expected_output - predict(input_array)

In [6]:
x = np.array(data.iloc[4])
x = x.astype(float)
INPUT = x
LAYER_1 = INPUT*LAYER_1_WEIGHTS
LAYER_1 = np.sum(LAYER_1,axis=1)
LAYER_2 = LAYER_1*LAYER_2_WEIGHTS
LAYER_2 = np.sum(LAYER_2,axis=1)
LAYER_2_SIG = sigmoid(LAYER_2)
LAYER_3 = LAYER_2_SIG*LAYER_3_WEIGHTS
LAYER_3 = np.sum(LAYER_3,axis=1)
LAYER_3_SIG = sigmoid(LAYER_3)
OUTPUT = LAYER_3_SIG*OUTPUT_WEIGHTS
OUTPUT = np.sum(OUTPUT,axis=1)
INPUT_SIZE = INPUT.size
OUTPUT[0]

0.8290780194939738

In [54]:
def OutputWeightDerivative(x):
    if(x<LAYER_3_SIZE):
        return LAYER_3_SIG[x]
    print('OUTPUT_WEIGHT_DERIVATIVE_INDEX_OOB')
    return 0

def Layer3WeightDerivative(x,y):
    if(x<LAYER_2_SIZE):
        if(y<LAYER_3_SIZE):
            d = OUTPUT_WEIGHTS[0][y]*LAYER_2_SIG[x]*sigmoidDerivative(LAYER_3[y])
            return d
    print('LAYER_3_WEIGHT_DERIVATIVE_INDEX_OOB')
    return 0

def Layer2WeightDerivative(x,y):
    if(x<LAYER_1_SIZE):
        if(y<LAYER_2_SIZE):
            s = 0
            for i in range(LAYER_3_SIZE):
                s = s+(OUTPUT_WEIGHTS[0][i]*LAYER_3_WEIGHTS[i][y]*sigmoidDerivative(LAYER_3[i]))
            return s*sigmoidDerivative(LAYER_2[y])*LAYER_1[x]
    print('LAYER_2_WEIGHT_DERIVATIVE_INDEX_OOB')
    return 0

def Layer1WeightDerivative(x,y):
    if(x<INPUT_SIZE):
        if(y<LAYER_1_SIZE):
            s = 0
            for i in range(LAYER_3_SIZE):
                d = 0
                for j in range(LAYER_2_SIZE):
                    d = d+(INPUT[x]*sigmoidDerivative(LAYER_2[j])*LAYER_3_WEIGHTS[i][j]*LAYER_2_WEIGHTS[j][y])
                s = s + (OUTPUT_WEIGHTS[0][i]*sigmoidDerivative(LAYER_3[i])*d)
            return s
    print('LAYER_1_WEIGHT_DERIVATIVE_INDEX_OOB')
    return 0
        
                

In [39]:
deriv1 = np.zeros((LAYER_1_SIZE,INPUT_SIZE))
for i in range(INPUT_SIZE):
    for j in range(LAYER_1_SIZE):
        deriv1[j][i] = Layer1WeightDerivative(i,j)

In [40]:
deriv2 = np.zeros((LAYER_2_SIZE,LAYER_1_SIZE))
for i in range(LAYER_1_SIZE):
    for j in range(LAYER_2_SIZE):
        deriv2[j][i] = Layer2WeightDerivative(i,j)

In [41]:
deriv3 = np.zeros((LAYER_3_SIZE,LAYER_2_SIZE))
for i in range(LAYER_2_SIZE):
    for j in range(LAYER_3_SIZE):
        deriv3[j][i] = Layer3WeightDerivative(i,j)

In [48]:
deriv4 = np.zeros((1,LAYER_3_SIZE))
for i in range(LAYER_3_SIZE):
    deriv4[0][i] = OutputWeightDerivative(i)

In [50]:
deriv4.shape

(1, 4)

In [11]:
ALPHA = 0.01
def train(x,EXPECTED_OUTPUT):
    e = error(x,EXPECTED_OUTPUT)
    deriv1 = np.zeros((LAYER_1_SIZE,INPUT_SIZE))
    for i in range(INPUT_SIZE):
        for j in range(LAYER_1_SIZE):
            deriv1[j][i] = Layer1WeightDerivative(i,j)
    deriv2 = np.zeros((LAYER_2_SIZE,LAYER_1_SIZE))
    for i in range(LAYER_1_SIZE):
        for j in range(LAYER_2_SIZE):
            deriv2[j][i] = Layer2WeightDerivative(i,j)
    deriv3 = np.zeros((LAYER_3_SIZE,LAYER_2_SIZE))
    for i in range(LAYER_2_SIZE):
        for j in range(LAYER_3_SIZE):
            deriv3[j][i] = Layer3WeightDerivative(i,j)
    deriv4 = np.zeros((1,LAYER_3_SIZE))
    for i in range(LAYER_3_SIZE):
        deriv4[0][i] = OutputWeightDerivative(i)
    global LAYER_1_WEIGHTS
    LAYER_1_WEIGHTS = LAYER_1_WEIGHTS + (ALPHA*e*deriv1)
    global LAYER_2_WEIGHTS
    LAYER_2_WEIGHTS = LAYER_2_WEIGHTS + (ALPHA*e*deriv2)
    global LAYER_3_WEIGHTS
    LAYER_3_WEIGHTS = LAYER_3_WEIGHTS + (ALPHA*e*deriv3) 
    global OUTPUT_WEIGHTS
    OUTPUT_WEIGHTS= OUTPUT_WEIGHTS + (ALPHA*e*deriv4)

In [10]:
for i in range(1000):
    x = np.array(data.iloc[i])
    x = x.astype(float)
    train(x,FGM[i])

NameError: name 'train' is not defined

In [60]:
s = 0
for i in range(5000):
    x = np.array(data.iloc[i])
    x = x.astype(float)
    s = s + abs(error(x,FGM[i]))

In [61]:
s/5000

0.47782473342195153

In [65]:
y = np.array([0,0,0,0,0,0,0,0,0,0,0,0])
y = y.astype(float)
z = [0.1,0.1,1,1,0.1,1,1,1,1,-1,-0.1,0.5]
for i in range(12):
    x = np.array([0.5,0.5,0,5,2,24,12,3,3,20,0.5,4])
    x = x.astype(float)
    p = predict(x)
    x[i] = x[i] + z[i]
    y[i] = y[i] + (predict(x)-p)
    x[i] = x[i] - z[i]

In [110]:
x = np.array([0,0,0,0,0,0,0,2,0,25,0,0])
x = x.astype(float)
predict(x)

0.29833441678879263

In [100]:
LAYER_2

array([0.50541722, 0.72165785, 0.05053444, 0.9058644 , 0.98582897,
       0.76385984])

In [27]:
data.head()

,HOME,WIN,FINAL_MARGIN,SHOT_NUMBER,PERIOD,MIN_LEFT,SHOT_CLOCK,DRIBBLES,TOUCH_TIME,SHOT_DIST,PTS_TYPE,CLOSE_DEF_DIST
0,0,1,24,1,1,37.15,10.8,2,1.9,7.7,0,1.3
1,0,1,24,2,1,36.23,3.4,0,0.8,28.2,1,6.1
2,0,1,24,3,1,36.00,0.0,3,2.7,10.1,0,0.9
3,0,1,24,4,2,35.78,10.3,2,1.9,17.2,0,3.4
4,0,1,24,5,2,34.57,10.9,2,2.7,3.7,0,1.1


In [2]:
FGM.head()

0    1
1    0
2    0
3    0
4    0
Name: FGM, dtype: int64

In [238]:
data.to_csv("SHOT_DATA.csv")

In [117]:
LAYER_1 = (np.random.rand(LAYER_1_COUNT,x.size)*2)-1

In [118]:
y

array([[-5.63452995e-01,  6.31205036e-01, -6.74895248e-01,
        -4.68656786e-01,  9.70603330e-01, -4.31018789e-01,
        -8.19071429e-01,  5.03685131e-01, -5.93828686e-01,
        -1.74606779e-01,  2.30669064e-01, -3.97306723e-01,
         9.13950182e-01,  9.46079526e-01,  6.90410017e-01,
        -2.39996000e-01,  6.94483090e-01,  6.41480366e-01,
        -8.91439872e-02, -3.84051548e-01],
       [-8.10388289e-01, -4.74211927e-01,  8.75234467e-02,
        -4.88948571e-01,  6.81424843e-01, -3.38348755e-01,
         7.24292261e-01, -4.38948662e-02,  6.34336795e-01,
         9.88570433e-01,  6.23529984e-01, -2.63742177e-01,
         3.14393702e-01, -1.82556197e-01, -7.79824884e-01,
        -5.23239565e-01,  8.03729179e-01,  6.54280490e-02,
        -1.92808270e-01, -6.91391467e-01],
       [-4.33127635e-01,  3.04977654e-01, -4.22590083e-01,
         8.39613444e-01,  8.63794627e-01,  4.84301677e-01,
         3.34542946e-01,  8.72479406e-01, -4.62752391e-01,
         9.71830963e-01,  7.2

In [138]:
data.head()

,FGM,FGA,FTM,FTA,TPM,TPA,ORB,DRB,AST,STL,TO,BLK,PTS,PFS
0,-0.092437,-0.269193,-0.846154,-0.869048,-1.0,-1.0,-0.826840,-0.205195,-0.867725,-0.455782,-0.476190,-0.555556,-0.247432,-0.365079
1,0.111765,0.055102,-0.515385,-0.450000,-1.0,-1.0,0.090909,0.084463,-1.000000,-0.428571,-0.725000,-1.000000,0.033333,-0.666667
2,-0.092437,-0.086492,-0.230769,-0.345238,-1.0,-1.0,-1.000000,-0.451712,-0.867725,-1.000000,-0.214286,-1.000000,-0.015873,0.587302
3,-0.164087,0.060150,-1.000000,-0.927632,-1.0,-1.0,-0.617225,-0.344672,-0.926901,-0.849624,-0.855263,-0.877193,-0.360165,-0.649123
4,-0.364706,-0.309388,-0.483077,-0.450000,-1.0,-1.0,-0.709091,-0.614413,-1.000000,-1.000000,-0.780000,-1.000000,-0.319216,-0.200000


In [234]:
data = data.drop(['SHOT_RESULT'],axis=1)

In [7]:
data.head()

,HOME,WIN,FINAL_MARGIN,SHOT_NUMBER,PERIOD,MIN_LEFT,SHOT_CLOCK,DRIBBLES,TOUCH_TIME,SHOT_DIST,PTS_TYPE,CLOSE_DEF_DIST
0,0,1,24,1,1,37.15,10.8,2,1.9,7.7,2,1.3
1,0,1,24,2,1,36.23,3.4,0,0.8,28.2,3,6.1
2,0,1,24,3,1,36.00,0.0,3,2.7,10.1,2,0.9
3,0,1,24,4,2,35.78,10.3,2,1.9,17.2,2,3.4
4,0,1,24,5,2,34.57,10.9,2,2.7,3.7,2,1.1


In [237]:
data['GAME_CLOCK'][0]

'1:09'